# Introduction to AI Virtual Characters

This notebook introduces the basic concepts of AI for Virtual Characters using the IAPV framework.

## Learning Objectives
- Understand basic vector mathematics for AI
- Create and manage virtual agents
- Implement simple steering behaviors
- Visualize agent movement patterns

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the src directory to Python path
sys.path.append(os.path.join(os.getcwd(), '..', 'src', 'python'))

from common.math_utils import Vector2D, Vector3D, Agent, Environment
from locomotion.steering_behaviors import FlockingAgent, SeekBehavior, WanderBehavior

## 1. Vector Mathematics

Virtual characters operate in 2D and 3D space, so understanding vectors is crucial.

In [ ]:
# Create and manipulate 2D vectors
position = Vector2D(5.0, 3.0)
velocity = Vector2D(2.0, -1.0)

print(f"Position: ({position.x}, {position.y})")
print(f"Velocity: ({velocity.x}, {velocity.y})")
print(f"Velocity magnitude: {velocity.magnitude():.2f}")

# Vector operations
new_position = position + velocity
print(f"New position after movement: ({new_position.x}, {new_position.y})")

# Normalize velocity
normalized_velocity = velocity.normalized()
print(f"Normalized velocity: ({normalized_velocity.x:.2f}, {normalized_velocity.y:.2f})")

## 2. Creating Virtual Agents

Let's create some basic virtual agents and observe their behavior.

In [ ]:
# Create an environment to hold our agents
environment = Environment()

# Create a simple wandering agent
agent = FlockingAgent("wanderer", Vector3D(0, 0, 0))
agent.steering_controller.clear_behaviors()
agent.steering_controller.add_behavior(WanderBehavior(weight=1.0))

environment.add_agent(agent)

print(f"Created agent '{agent.id}' at position {agent.position.to_tuple()}")

## 3. Simulating Agent Movement

Let's simulate the agent's movement over time and track its path.

In [ ]:
# Simulation parameters
dt = 0.1  # Time step
steps = 200  # Number of simulation steps

# Track agent positions
positions_x = []
positions_y = []

# Run simulation
for step in range(steps):
    # Update environment
    environment.update(dt)
    
    # Record position
    pos = agent.position
    positions_x.append(pos.x)
    positions_y.append(pos.z)  # Use z for 2D visualization

print(f"Simulation completed. Agent moved through {len(positions_x)} positions.")

## 4. Visualizing Agent Behavior

Let's visualize the agent's movement pattern.

In [ ]:
# Create a plot of the agent's path
plt.figure(figsize=(10, 8))

# Plot the path
plt.plot(positions_x, positions_y, 'b-', alpha=0.7, linewidth=2, label='Agent Path')

# Mark start and end positions
plt.plot(positions_x[0], positions_y[0], 'go', markersize=10, label='Start')
plt.plot(positions_x[-1], positions_y[-1], 'ro', markersize=10, label='End')

# Add some style
plt.grid(True, alpha=0.3)
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Wandering Agent Movement Pattern')
plt.legend()
plt.axis('equal')

plt.tight_layout()
plt.show()

# Show some statistics
total_distance = sum([
    np.sqrt((positions_x[i+1] - positions_x[i])**2 + (positions_y[i+1] - positions_y[i])**2)
    for i in range(len(positions_x) - 1)
])

print(f"Total distance traveled: {total_distance:.2f} units")
print(f"Average speed: {total_distance / (steps * dt):.2f} units/second")

## 5. Seek Behavior

Now let's create an agent that seeks towards a target.

In [ ]:
# Create a new environment
environment2 = Environment()

# Create a seeking agent
seeker = FlockingAgent("seeker", Vector3D(-20, 0, -20))
target_position = Vector3D(20, 0, 20)

seeker.steering_controller.clear_behaviors()
seeker.steering_controller.add_behavior(SeekBehavior(target_position, weight=1.0))

environment2.add_agent(seeker)

# Track positions for both agents
seeker_x, seeker_y = [], []

# Run simulation
for step in range(100):
    environment2.update(dt)
    
    pos = seeker.position
    seeker_x.append(pos.x)
    seeker_y.append(pos.z)

# Visualize
plt.figure(figsize=(10, 8))

plt.plot(seeker_x, seeker_y, 'r-', linewidth=2, label='Seeker Path')
plt.plot(seeker_x[0], seeker_y[0], 'go', markersize=10, label='Start')
plt.plot(target_position.x, target_position.z, 'bs', markersize=12, label='Target')
plt.plot(seeker_x[-1], seeker_y[-1], 'ro', markersize=10, label='Final Position')

plt.grid(True, alpha=0.3)
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Seek Behavior - Agent Moving Towards Target')
plt.legend()
plt.axis('equal')

plt.tight_layout()
plt.show()

final_distance = np.sqrt((seeker_x[-1] - target_position.x)**2 + (seeker_y[-1] - target_position.z)**2)
print(f"Final distance to target: {final_distance:.2f} units")

## 6. Exercise: Multiple Agents with Flocking

Try creating multiple agents and observe their flocking behavior.

In [ ]:
# Exercise: Create 5 flocking agents and observe their behavior
environment3 = Environment()

# Create multiple agents
agents = []
for i in range(5):
    # Random starting positions
    x = np.random.uniform(-10, 10)
    y = np.random.uniform(-10, 10)
    
    agent = FlockingAgent(f"boid_{i}", Vector3D(x, 0, y))
    agents.append(agent)
    environment3.add_agent(agent)

# Track all agent positions
all_paths = {agent.id: {'x': [], 'y': []} for agent in agents}

# Simulation
for step in range(300):
    # Update neighbor relationships for flocking
    for agent in agents:
        neighbors = environment3.get_agents_in_radius(agent.position, 15.0)
        agent.set_neighbors(neighbors)
    
    environment3.update(dt)
    
    # Record positions
    for agent in agents:
        all_paths[agent.id]['x'].append(agent.position.x)
        all_paths[agent.id]['y'].append(agent.position.z)

# Visualize flocking behavior
plt.figure(figsize=(12, 10))

colors = ['red', 'blue', 'green', 'orange', 'purple']
for i, agent in enumerate(agents):
    path = all_paths[agent.id]
    plt.plot(path['x'], path['y'], color=colors[i], alpha=0.7, linewidth=2, label=f'Agent {i+1}')
    plt.plot(path['x'][0], path['y'][0], 'o', color=colors[i], markersize=8)
    plt.plot(path['x'][-1], path['y'][-1], 's', color=colors[i], markersize=8)

plt.grid(True, alpha=0.3)
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Flocking Behavior - Multiple Agents')
plt.legend()
plt.axis('equal')

plt.tight_layout()
plt.show()

print("Observe how the agents start to move together as a group (flock) over time!")

## Summary

In this notebook, you've learned:

1. **Vector Mathematics**: The foundation of all AI movement and spatial reasoning
2. **Agent Creation**: How to create and manage virtual characters
3. **Steering Behaviors**: Basic movement patterns like wandering and seeking
4. **Simulation**: How to run and visualize AI behavior over time
5. **Flocking**: Emergent group behavior from simple individual rules

## Next Steps

Try these exercises to deepen your understanding:

1. Modify the wander behavior parameters and observe the changes
2. Create agents that flee from a threat
3. Implement obstacle avoidance
4. Experiment with different flocking parameters
5. Create a simple predator-prey simulation

Continue with the next notebook: **02_Steering_Behaviors.ipynb** to learn more advanced movement patterns!